In [1]:
import tensorflow as tf
import numpy as np

In [10]:
# ported from caffe: https://github.com/BVLC/caffe/blob/master/examples/mnist/lenet_train_test.prototxt
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28, 1),
                           filters=20,
                           kernel_size=5),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=50,
                         kernel_size=5),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Softmax()])

In [17]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

model.compile(optimizer=tf.train.GradientDescentOptimizer(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=x_train, y=y_train, epochs=1)

Epoch 1/1
60000/60000 [==============================] - 367s 6ms/step - loss: 1.1558 - acc: 0.5922
